# Data-sources exploration using `eo-learn`

This notebook shows some examples on how to retrieve EO and non-EO data using `eo-learn`. 

The steps are as follow:
 * split area of interest into easy-to-process EOPatches
 * add Sentinel-2 imaging data
 * add Sentinel-1 imaging data

Add generic packages

In [ ]:
%matplotlib inline

import os
from pathlib import Path

from matplotlib import dates
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shapely.geometry import Polygon, box, shape, mapping
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np

Set path to data

In [ ]:
data_dir = Path('..', 'data')
os.listdir(data_dir)

`eo-learn` and `sentinelhub` imports

In [ ]:
from eolearn.core import EOTask, EOPatch, FeatureType
from eolearn.io import SentinelHubInputTask, SentinelHubEvalscriptTask
from eolearn.geometry import VectorToRasterTask

In [ ]:
from sentinelhub import BBoxSplitter, BBox, CRS, DataCollection

## 1. Split country into smaller bounding boxes <a id='splitter'></a>

Load shapefile of Denmark

In [ ]:
country_filename = data_dir / 'denmark.geojson'
country = gpd.read_file(country_filename)

country.plot()
country.crs

Set CRS to UTM

In [ ]:
country_crs = CRS.UTM_32N
country = country.to_crs(country_crs.pyproj_crs())

country.plot()
country.crs

Get size of country in pixels to decide number and size of bounding boxes

In [ ]:
country_shape = country.geometry.values[-1]

width_pix = int((country_shape.bounds[2] - country_shape.bounds[0]) / 10)
height_pix = int((country_shape.bounds[3] - country_shape.bounds[1]) / 10)

print(f'Dimension of the area is {width_pix} x {height_pix} pixels')

Split area into 45x35 boxes bounding 

In [ ]:
bbox_splitter = BBoxSplitter([country_shape], country_crs, (45, 35))

In [ ]:
geometry = [bbox.geometry for bbox in bbox_splitter.get_bbox_list()]
bbox_list = bbox_splitter.get_bbox_list()
idxs_x = [info['index_x'] for info in bbox_splitter.get_info_list()]
idxs_y = [info['index_y'] for info in bbox_splitter.get_info_list()]

gdf = gpd.GeoDataFrame(
    {'index_x':idxs_x, 'index_y':idxs_y},
    geometry=[bbox.geometry for bbox in bbox_list],
    crs=bbox_list[0].crs.pyproj_crs()
)

gdf.head()

Plot results

In [ ]:
# if bboxes have all same size, estimate offset
xl, yl, xu, yu = gdf.geometry[0].bounds
xoff, yoff = (xu - xl) / 3, (yu - yl) / 5

# figure
fig, ax = plt.subplots(figsize=(45,35))
gdf.plot(ax=ax, facecolor='w', edgecolor='r', alpha=0.5, linewidth=2)
country.plot(ax=ax, facecolor='w', edgecolor='b', alpha=0.5, linewidth=2.5)
ax.set_title('Denmark tiled in a 45 x 35 grid');

# add annotiation text
fontdict = {'family': 'monospace', 'weight': 'normal', 'size': 14}
for idx in gdf.index:
    eop_name = '{0}x{1}'.format(gdf.index_x[idx], gdf.index_y[idx])
    centroid, = list(gdf.geometry[idx].centroid.coords)
    ax.text(centroid[0] - xoff, centroid[1] + yoff, str(idx), fontdict=fontdict)
    ax.text(centroid[0] - xoff, centroid[1] - yoff, eop_name, fontdict=fontdict)

## 2. Retrieve S2 L1C data <a id="sentinel-2"></a>

In [ ]:
s2_rgb_task = SentinelHubInputTask(
    data_collection=DataCollection.SENTINEL2_L1C,
    bands=['B04', 'B03', 'B02'],
    bands_feature=(FeatureType.DATA, 'S2-RGB'),
    additional_data=[(FeatureType.MASK, 'dataMask')],
    resolution=(10, 10),
    maxcc=0.1
)

In [ ]:
ndvi_evalscript = """
//VERSION=3

function setup() {
  return {
    input: ["B04", "B08"],
    output:[
      {
        id: "ndvi",
        bands: 1,
        sampleType: SampleType.FLOAT32
      },
    ]
  }
}

function evaluatePixel(sample) {
  let ndvi = index(sample.B08, sample.B04);
  return {
    ndvi: [ndvi],
  };
}
"""

s2_ndvi_task = SentinelHubEvalscriptTask(
    features=[(FeatureType.DATA, 'ndvi', 'NDVI')],
    evalscript=ndvi_evalscript,
    data_collection=DataCollection.SENTINEL2_L1C,
    resolution=(10, 10),
    maxcc=0.1
)

In [ ]:
time_interval = ['2019-05-01','2019-09-01']
idx = 436
bbox = bbox_splitter.bbox_list[idx]

Download TRUE-COLOR

In [ ]:
eopatch = s2_rgb_task.execute(bbox=bbox, time_interval=time_interval)

eopatch

Download NDVI

In [ ]:
eopatch = s2_ndvi_task.execute(eopatch)

eopatch

In [ ]:
eopatch.timestamp

Plot RGB of time frames

In [ ]:
time_idx = 0

rgb = eopatch.data['S2-RGB']

fig, ax = plt.subplots(figsize=(15, 15))
ax.imshow(rgb[time_idx] * 3.5);

Plot the median RGB values

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))

ax.imshow(np.median(rgb, axis=0).squeeze() * 3.5);

Plot the median NDVI values

In [ ]:
ndvi = eopatch.data['NDVI']
median_ndvi = np.median(ndvi, axis=0).squeeze()

fig, ax = plt.subplots(figsize=(15,15))
im = ax.imshow(median_ndvi, cmap=plt.cm.YlGn)

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical');

Plot temporal NDVI of a given location

In [ ]:
dates_num = dates.date2num(eopatch.timestamp)
dates_str = [timestamp.date().isoformat() for timestamp in eopatch.timestamp]

fig, ax = plt.subplots(figsize=(15, 15))
ax.plot(dates_num, ndvi[:, 100, 550, :].squeeze(), 'g')

ax.set_title('NDVI evolution')
ax.set_xticks(dates_num);
ax.set_xticklabels(dates_str, rotation=45, ha='right');
ax.set_ylabel('NDVI');

## 4. Retrieve S1 data <a id="sentinel-1"></a>

In [ ]:
s1_iw_des_task = SentinelHubInputTask(
    data_collection=DataCollection.SENTINEL1_IW_DES,
    bands=['VV'],
    bands_feature=(FeatureType.DATA, 'S1-IW-DES'),
    additional_data=[(FeatureType.MASK, 'dataMask')],
    resolution=(10, 10)
)

s1_iw_asc_task = SentinelHubInputTask(
    data_collection=DataCollection.SENTINEL1_IW_ASC,
    bands=['VV'],
    bands_feature=(FeatureType.DATA, 'S1-IW-ASC'),
    additional_data=[(FeatureType.MASK, 'dataMask')],
    resolution=(10, 10)
)

In [ ]:
eopatch_s1_des = s1_iw_des_task.execute(bbox=bbox, time_interval=['2019-07-01','2019-08-01'])

eopatch_s1_des

[VV-polarised Timescan Composite](https://github.com/ESA-PhiLab/OpenSarToolkit/blob/master/README.md)

In [ ]:
vv_data = eopatch_s1_des.data['S1-IW-DES']
vv_data[np.isnan(vv_data)] = 0

vv_des_r = np.percentile(vv_data, 80, axis=0)
vv_des_g = np.percentile(vv_data, 20, axis=0)
vv_des_b = np.std(vv_data, axis=0)

vv_rgb = np.concatenate((vv_des_r, vv_des_r, vv_des_b), axis=-1)

plt.figure(figsize=(15, 15))
plt.imshow(vv_rgb);

In [ ]:
eopatch_s1_asc = s1_iw_asc_task.execute(bbox=bbox, time_interval=['2019-07-01','2019-08-01'])

eopatch_s1_asc

In [ ]:
vv_data = eopatch_s1_asc.data['S1-IW-ASC']
vv_data[np.isnan(vv_data)] = 0

vv_des_r = np.percentile(vv_data, 80, axis=0)
vv_des_g = np.percentile(vv_data, 20, axis=0)
vv_des_b = np.std(vv_data, axis=0)

vv_rgb = np.concatenate((vv_des_r, vv_des_r, vv_des_b), axis=-1)

plt.figure(figsize=(15, 15))
plt.imshow(vv_rgb);

In [ ]:
vv_data.shape

In [ ]:
vv_data = eopatch_s1_asc.data['S1-IW-ASC']
vv_data[np.isnan(vv_data)] = 0

vv_des_r = vv_data[0]
vv_des_g = vv_data[2]
vv_des_b = vv_data[10]

vv_rgb = np.concatenate((vv_des_r, vv_des_r, vv_des_b), axis=-1)

plt.figure(figsize=(15, 15))
plt.imshow(vv_rgb);

Similarly, Sentinel-2 L2A data can be added, as well as Digital Elevation data